In [1]:
import os

In [2]:
%pwd

'e:\\FullStack_Data\\MACHINE_LEARNING\\PROJECTS\\junk\\DL_Proj5\\research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'e:\\FullStack_Data\\MACHINE_LEARNING\\PROJECTS\\junk\\DL_Proj5'

In [ ]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class PreparaBaseModelConifig:
    root_dir: Path
    base_model_path: Path
    updated_base_model_path: Path
    params_IMAGE_SIZE: list
    params_LEARNING_RATE: float
    params_include_top: bool
    params_WEIGHTS: str
    params_classes: int

In [ ]:
from src.Dl_Project5.utils.common import create_directories,load_yaml
from src.Dl_Project5.constants import *

class ConfigurationManager:
    def __init__(self,config_filepath=CONFIG_FILE_PATH,params_file_path=PARAMS_FILE_PATH):
        self.config=load_yaml(config_filepath)
        self.params=load_yaml(params_file_path)

        create_directories([self.config.artifacts_root])

    def get_prepara_basemodel_config(self)-> PreparaBaseModelConifig:
        create_directories([self.config.root_dir])

        prepare_base_model=PreparaBaseModelConifig(
            root_dir: self.config.root_dir
            base_model_path: self.config.base_model_path
            updated_base_model_path: self.config.updated_base_model_path
            IMAGE_SIZE: self.params.params_IMAGE_SIZE
            LEARNING_RATE: self.params.params_LEARNING_RATE
            params_include_top: self.params.params_include_top
            WEIGHTS: self.params.params_WEIGHTS
            params_classes: self.params.params_classes

        )

        return prepare_base_model

In [ ]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf

class PrepareBaseModel:
    def __init__(self, config: PrepareBaseModelConfig):
        self.config = config
    
    def get_base_model(self):
        self.model = tf.keras.applications.vgg16.VGG16(
            input_shape=self.config.params_image_size,
            weights=self.config.params_weights,
            include_top=self.config.params_include_top
        )

        self.save_model(path=self.config.base_model_path, model=self.model)

    @staticmethod
    def _prepare_full_model(model, classes, freeze_all, freeze_till, learning_rate):
        if freeze_all:
            for layer in model.layers:
                model.trainable = False
        elif (freeze_till is not None) and (freeze_till > 0):
            for layer in model.layers[:-freeze_till]:
                model.trainable = False

        flatten_in = tf.keras.layers.Flatten()(model.output)
        prediction = tf.keras.layers.Dense(
            units=classes,
            activation="softmax"
        )(flatten_in)

        full_model = tf.keras.models.Model(
            inputs=model.input,
            outputs=prediction
        )

        full_model.compile(
            optimizer=tf.keras.optimizers.SGD(learning_rate=learning_rate),
            loss=tf.keras.losses.CategoricalCrossentropy(),
            metrics=["accuracy"]
        )

        full_model.summary()
        return full_model

    def update_base_model(self):
        self.full_model = self._prepare_full_model(
            model=self.model,
            classes=self.config.params_classes,
            freeze_all=True,
            freeze_till=None,
            learning_rate=self.config.params_learning_rate
        )

        self.save_model(path=self.config.updated_base_model_path, model=self.full_model)

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)

In [ ]:
try:
    config = ConfigurationManager()
    prepare_base_model_config = config.get_prepare_base_model_config()
    prepare_base_model = PrepareBaseModel(config=prepare_base_model_config)
    prepare_base_model.get_base_model()
    prepare_base_model.update_base_model()
except Exception as e:
    raise e